In [1]:
!ls "drive/My Drive/bert_wiki_ja"

bert_check_point.model	  model.ckpt-1400000.data-00000-of-00001  wiki-ja.model
bert_config.json	  model.ckpt-1400000.index		  wiki-ja.vocab
bert_wiki_ja_model.ipynb  model.ckpt-1400000.meta


In [2]:
!pip install keras-bert
!pip install sentencepiece
!pip install mecab-python3

  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=9e234e3f01ca8d1473d157c37907ddd1aca7fa6527981c86f4251c5ffb5797cb
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for keras-transformer: filename=keras_transformer-0.32.0-cp36-none-any.whl size=13266 sha256=54497311bda5b3098208a1f2009a5e5f59b8c4599d2f849e1a9969e071a87e22
  Stored in directory: /root/.cache/pip/wheels/62/f0/ce/82fa5d024d5ef8e263f26a50dcee23820efe245680ce9c922a
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=c58503eb86913c99ff0da25e239404a31a085eba2bd1be38e199df0fb750b9b9
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=29129d3d9c783cde5f3ff34564f8b9871a2cebf2465245b004e8766404e39ac8
  Stor

In [3]:
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import AdamWarmup, calc_train_steps
from keras_bert import get_custom_objects
from keras import utils, Input, Model
from keras.layers import Dense, Dropout, LSTM, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model

import sentencepiece as spm
import pandas as pd
import numpy as np
import re
import MeCab
mecab = MeCab.Tagger("-Ochasen")

Using TensorFlow backend.


In [0]:
data = [
        ['留学先に行った時、最初はなかなか英語で会話ができず、悔しい思いをしました。自分自身を振り返った時、日本では周りの人に自分から話しかけるということはしていませんでした。英語ができないから会話ができないのではなく、自分から行動をしていないことに気づくことができたのです。それに気づいてからは、積極的に自分から声をかけるようにしました。半年後には、英語での日常会話ができるようになり、日本に帰ってからも積極的に周りに話しかけられるようになりました。', 1],
        ['数々のグループワークを体験し、その経験から、私は協調性と行動力に自信があります。学業において、数人の学生と組まされ、様々な課題を達成することが非常に多く、またほとんどの場合が全員初対面のメンバーで組まされるため、私は積極的に周りを巻き込みながら課題に取り組んでおりました。ただし、リーダーシップを必ずしも取るわけではありません。メンバーの年齢やバックグランドを把握してから、誰がリーダーシップをとるべきかを、まず吟味します。自分以外に指揮を取るべき人がいればその人をサポートするような役回りをします。自分が指揮をとるにしても、取らないにしても、全員を会話に巻き込みグループとして親和性を高めます。親和性を高めることで、全員がより積極性を持って行動するようになると考えるているからです。こういった行動を取ることで、私はグループワークにおいて高評価を得ることが多く、こういった行動が成績に直結したと考えています。', 1],
        ['私の売りはチーム活性化のために率先して動く行動力です。特にこの特徴が顕著に出ていたのはサークル活動で新入生歓迎活動の代表を任された時です。私の所属する広告サークルは毎年100名弱サークルに入会するため春の新入生歓迎活動に力を入れており、フリーペーパーは広報活動として重要なものでした。しかし作成時期が春休みということもあり人手不足による作業量の過多が発生しました。そこで、私はそれぞれに連絡を取り、フリーペーパーの作成にあたって、自分は企画や制作などの工程のうちどのような作業をしたいか希望を取り、それぞれに負担がかからないような作業を分散するようにしました。また、冊子の中身も手に取る新入生に向けて活動の紹介や活動の雰囲気だけでなく大学紹介のページや周辺地域の特集を組むなどこれまでにない新たな冊子の構成の話し合いにも力を入れました。その結果納期に間に合い1500部の冊子を経由して180人の新入生を獲得することに成功しました。作っていく中で地道ながらも何度もサークル員と連絡を取り続けたことが、活動の集大成としての面もある冊子がサークル員の個性を生かしたものになったのではないかと思います。', 1],
        ['わたしは気配りがよくできる人間だとよく言われます。現在飲食店でアルバイトをしていますが、お客さんへの接客サービスは良く言われることが多く、ありがとうの言葉を度々いただきます。自らでこうされたらうれしい、便利かなと思うことを常に念頭に置いて、接客を行います。また、大学での部活動での打ち上げやイベントなどの幹事を行っており、お店選びや、内容などを大多数の希望をくみ上げ、みんなで楽しむ部活動をしてきました。他人を喜ばせるという行為は非常に難しく、間違えたことや出過ぎた気づかいは相手を怒らせたり、不快に思わせたりします。しかし、誰かを喜ばせることはその人だけでなく自分自身も喜ぶことが出来、その喜びを共有することができるので私はそういった気遣いをすることに誇りを感じます。この気遣いを仕事で積極的に取り入れ、自分の仕事だけでなく、いろんな人が気持ちよく仕事ができるようにしたいと思います。', 1],
        ['私はサークルで企画長として活動していました。始めた頃はイベントの参加率が6割程で、いまいちまとまりに欠ける状態でした。そこで部員のニーズを掴むために、アンケートを実施し、積極的にヒアリングを行うなど、自分なりに行動しました。その結果を踏まえて球技大会、飲み会などを開催し、参加率も次第に上昇、最終的には常に9割をキープするまでになったのです。社会に出ても顧客のニーズに基づいた提案型の営業マンになりたいと考えています。', 1],
        ['私はコツコツと努力ができる人間です。すぐに成果が出なくても、自身を奮い立たせ、コツコツと努力を積み上げ、目標を達成することが出来ます。具体的に、それを発揮したのがTOEICの点数を400点から800点にアップさせた経験です。将来は海外で働きたいという思いがあり、TOEICの学習を始めました。バイト・学業で時間がない中で点数をあげるため、1.学習の範囲を細分化し（今日は不定詞の完了形を学ぶ等）、毎日決めた範囲を確実にこなした。2.スキマ時間を活用するため、文法の問題集を持ち歩いて、10分空き時間があれば勉強をするようにした。3.移動時間を無駄にしないため、電車ではリスニングの学習を進めた。この3つを実践し、学習を続けました。毎日小さな努力を積み上げた結果、上記の成果をあげることができました。', 1],
        ['私の強みは、行動力にあると考えています。大学2年生の8月から大学3年生の12月まで、不動産の会社でインターン生として投資用の中古物件の販売を経験しました。当初はアポを取ることもできず、営業成績は12人いた同期のインターン生の中でも最下位でした。しかしこのままではいけないと思い、自分の課題・弱点をリストアップして、それらを克服するための行動を起こしていきました。具体的には、①営業やコミュニケーション系のセミナーへの参加。②関連書籍の読み込み。③社内の先輩や上司の方々にアドバイスを求める。この3点を地道に継続したことで、自分の課題・弱点を一つずつ潰すことができ、営業力を着実に向上していきました。その結果、1年後には同期の中でTOPの営業成績を修めることができました。この経験を通して、課題解決のために現状と理想の差を分析して、その差を埋めるための具体的な行動を起こすことの重要性を学びました。貴社の業務においても、課題に対する打ち手を冷静に分析し、問題解決や目標達成のための行動を自発的に起こせればと考えております。', 1],
        ['私の長所は、相手の立場に立って物事を考えられる点です。私は所属するゼミナールでの会議で、ファシリテーターをしていました。その際に、なかなか意見を言い出せないメンバーを見極め、その人にこちらから意見を聞くようにしていました。その結果、その人の意見から討論が活発になり実りある会議を行う事ができました。この経験から、相手のことを考えて行動することで、より良い組織を構築できることを学びました。また、こうした相手の気持ちを見抜くスキルは御社の社内会議でも活かす事ができると考えています。', 1],
        ['私の長所は、相手の気持ちを汲み取って行動できる点です。長所を活かした経験に、アルバイトでの出来事が挙げられます。私は飲食店でアルバイトをしていますが、その際にお客さまが求めていることを先読みして行動する事ができます。例えば、何も言われずとも取り皿を追加で出したり、メニューに迷っていたらこちらから声をかけると言ったことです。その結果、お客さんには「こちらの考えを感じ取ってくれていて有難い」と褒めていただいたり、中には私がいるからまた来ると言ってくださる常連の方までできました。こうしたことは、御社に入った際に顧客対応の面で活かす事ができると考えています。', 1],
        ['私の長所は、相手が何を考えているかを感じ取り行動できる点です。その例として、塾講師のアルバイトで生徒を指導した経験が挙げられます。生徒のやる気やテストの出来具合はまちまちです。そのため、塾講師として1人ひとりに合わせた指導を行うことが求められます。そこで私は「なぜこの子は勉強が嫌いなのか」や「どうして点数が伸びないのか」などを、生徒の立場に立って物事を考えてきました。このように相手の立場に立って考えるようにした結果、担当していた生徒が徐々に勉強にやる気を持ち始め、テストの点も右肩上がりになりました。この経験から、何かを伝える際にはまず相手の気持ちを汲み取る事が重要なのだと学びました。私が御社に入った際にも、まずは相手の気持ちを考えることから始め、顧客や社員の方とのリレーションシップを築いていきたいと考えています。', 1],
        ['私の強みは、気取らず人に親しみやすさを与えられることです。私は大学の英会話サークルで新入生歓迎会部長を担当していました。新入生が集まる場では、新入生同士が仲良くなれるよう、共通の話題を見つけだし親近感を沸かせました。また、自分のサークルのいい点だけでなく、改善点を相談するなど、ありのままの実態をさらけ出すことで、サークルの本質的な紹介にも成功しました。その結果、入部数は昨年度を大きく上回ることができました。私の気取らず、人に親しみやすさを与えるこの強みは、貴社の営業部門で新規顧客開拓時にも貢献できると考えます。', 1],
        ['私はある和食バイキングレストランでアルバイトをしておりました。その店は焼酎とワインをウリにしており、一方で私はカクテルメニューに魅力がないと感じておりました。そこで私は本を買い、知り合いのバーテンに相談して、今ある材料からできるカクテルのリストを作り、店長にカクテルの種類を増やすこととオリジナルカクテルを加えることを提案しました。結果、リストの中からいくつかを採用して頂き、オリジナルカクテルについては開発から社長へのプレゼンまでも任せられ、最終的には試行錯誤の末の４つの試作品から１つを採用して頂くことに成功しました。それ以来カクテルの売り上げは伸び、オリジナルカクテルは人気商品となりました。私はお客様や店長からお褒めの言葉を頂き、大きな喜びを感じました。この経験から提案することの意義を知るとともに、その難しさを体験しました。これからも分析を提案に変えることを続けていきたいと思います。', 1],
        ['私は、自分のポジションを俯瞰し組織に貢献することができます。この強みを最も発揮したのがゼミの研究発表会です。ゼミの研究発表会はチーム毎に行われるため、誰か一人がリーダーとしてチームをまとめる必要がありました。私は、元々みんなを引っ張っていくタイプではありません。しかし他のメンバーは部活動に所属しており、ゼミ活動にあまり時間が割けないと考えました。そこでリーダーに立候補し、研究会に向けての準備を進めました。周りのメンバーが時間を取れない分、率先して資料の作成を引き受けたり、スケジュールの調整を行ったりしました。初めてのリーダー経験ということもあり不安でしたが、無事研究発表会を終え、教授方からも高評価をいただけました。貴社に入社した後も、組織全体の中で自分ができる役割を見つけ行動することで、貴社の目標達成に貢献したいと考えております。', 1],
        ['私は、相手の立場で物事を考え行動に移すことができます。この強みを最も発揮したのが大学時代のゼミ活動です。ゼミ活動では、中山間地域の地域活性化活動に取り組んできました。限界集落に赴き、現地の人の話を聞く中で、稲を収穫する際の人出が足りないことが分かりました。そこで、私は自分にできることは何かを考え、1週間泊まり込みで収穫作業を手伝うことにしました。集落の皆さんと協力して作業を行なった結果、期日内に稲の収穫を終えることができたのです。全ての作業を終えた後に集落の皆さんと食べたご飯の味は忘れられません。この経験から、相手の求めていることを考え行動に移すことの大切さを確信できました。入社後も常にお客様の立場で物事を考え、最適な提案を行うことで貴社に貢献します。', 1],
        ['私の強みは粘り強さです。ゼミのグループ研究で不参加のメンバーがおり、リーダーの私は頭を悩ませていました。「人を動かすために大事なことは、相手ではなく自分を変えること」とよくいわれますが、私は態度を変えました。一方的な出席要請をやめ、こまめに連絡を取っては、彼の興味を引く研究材料の話題で盛り上げ続けたところ、彼が自らの意思で意欲的に加わるようになったのです。メンバー1名抜きでも進行は可能でしたが、力を尽くした結果、全員で密度の濃い発表ができました。入社後も、諦めない気持ちを大切に業務を遂行いたします。', 1],
        ['私は昔から、自分の意見をはっきりと主張するタイプです。そのためか、高校では生徒会役員と部長を任され、大学時代のアルバイト先ではバイトリーダーを任されました。これらの経験から、信頼されるリーダーに必要なのは自分の意見を押し付ける強引さではなく、人の意見に耳を傾ける姿勢と対話力だと学びました。リーダーの大変さを知る私だからこそ、リーダーである先輩方の手助けとなる仕事を率先して行いたいと思います。そして、いずれは自分がリーダーシップを発揮できる仕事をしたいと考えています。', 1],
        ['私は任されたことをこなすだけでなく、それ以上の成果を出すように努めています。アルバイト先の店舗で商品の発注を任されているのですが、それをミスなくこなすだけでなく、より効率的な方法を模索しました。高い頻度で注文する商品の品名と商品コードのリストを作り、その都度カタログから探す手間を省いた結果、発注にかかっていた時間を40分から15分に短縮することができました。このように、御社でも求められた以上の成果を出せるよう努めて参ります。', 1],
        ['私は物怖じせずに挑戦することができます。大学1年生の時、教養科目で履修した「手話」に興味を持ち、友人4人と福祉ボランティアのサークルを立ち上げました。手話や点字の学習会や福祉施設での交流会を行ったり、募金のためのチャリティイベントを行ったりしました。その結果、初回のチャリティで8万円が集まり、福祉施設に寄付することができました。このことから可能性に踏み出すことで、自分でも思ってみなかったことが成し遂げられると知りました。御社においても「物怖じしない」をモットーに、営業として活躍したいと考えております。', 1],
        ['私はチームのメンバーが最大限のパフォーマンスを発揮できるようにサポートできます。学園祭で演劇の大道具担当になったときに、手先があまり器用でない私は、裏方のさらに裏方に徹し、資材を補充したり、筆を洗って乾かしたりといった「制作以外の雑務」を引き受けました。当時は「自分にはこれくらいしかできない」という後ろ向きな気持ちがありましたが、ある日大道具のリーダーから私のおかげで制作に集中できると言われ、自分が役に立てているのだと自信が持てました。このサポート力を御社の営業事務で活かしたいと考えております。', 1],
        ['私は、何事にも妥協せず上を目指して全力を尽くします。高校時代バレー部に所属していた私は、体格に恵まれたおかげかすぐにレギュラーに選ばれました。しかし、選ばれたことに慢心せず、誰よりも早くに練習を開始し、毎日最後まで残ってサーブの練習を続けました。その結果、試合では必ずと言っていいほどサービスエースで得点することができ、2年生のときには部内45名の中で年間MVPに選ばれました。この向上心だけでなく、それを支える継続力と実行力は、御社に貢献できるものと確信しております。', 1],
        ['私はスポンジのような吸収力があり、些細なことでも吸収して成長することが出来ます。ゼミでは地域社会の活性化について学びましたが、最初は何の知識もなく分からないことだらけで、チームの足を引っ張ってばかりでした。私はそのことに悔しさを感じ、チームに迷惑をかけないためにも猛勉強をしました。独学で学ぶだけではなく、少しでも気になったところは教授に質問し、貪欲に学ぶことでチームでも活躍し、大学最後の発表会を成功に導くことが出来ました。私は御社でも素直に学ぶ姿勢を持ち、何でも貪欲に吸収し、企業に貢献出来るように成長していきたいと考えています。', 1],
        ['私は周囲の人と力を合わせながらチームを盛り上げることが出来ます。大学時代のゼミでは年末に発表会があり、それに向けて日々研究を続けていました。発表はチームごとに行われますが、私のチームはやる気もなく研究がなかなか進みませんでした。これではいけないと思い、誰が何をすべきかを明確にして指示を出し、私自身も一生懸命に研究に取り組むことでチームを鼓舞しました。熱意が伝わり、そこからしっかりと研究を進めることができ、発表会でも高い評価を得ることが出来ました。私は御社でもチームをサポートし、盛り上げていくことでパフォーマンスを高め、チーム全体での成功を目指すことで利益に貢献したいと考えています。', 1],
        ['私の長所は、集中力があることです。私は高校生の頃、将棋をしていました。学校では練習相手がいたので問題ありませんでしたが、家には将棋が打てる人がいなかったので練習することができませんでした。そこで私は、長所である集中力を活かし、イメージトレーニングをすることにしました。実際に手を動かさないと練習にならないと思っていましたが、次の日から打ち方が全然違うことを実感しました。それからは毎日1時間のイメージトレーニングをおこない、最後の大会では県1位の成績で全国大会に出場しました。貴社でも私の集中力を活かし、成長するために必要な手法をいち早く見つけ貢献していきたいと思います。', 1],
        ['私は、人よりも集中力があると自負しています。そう感じた経験は、中学生のときです。私は本を読むことが好きで、放課後は毎日図書館に行っていました。図書館では時間が経つのも忘れ、1日2～3冊のペースで読んでいました。私は別段勉強ができた訳ではないですが、本を読むことで様々な知識を得ることができました。その知識が、高校受験や全国模試などで活きたと思っています。もし貴社に入社致しましたら、集中して仕事をこなしながら、新しい知識や技術を吸収していきたいと思っております。', 1],
        ['私には、高い集中力があります。そう感じたきっかけは、スーパーでのアルバイトです。経験がある品出しを手際よくこなしていると、店長から「集中力があるね」と褒められたこともあります。しかし、集中して取り組むあまり、レジ対応のアナウンスを聞き逃してしまうことがありました。そのことで、集中しながらも周りの状況を把握しなければいけないと痛感することができました。それからは、自分の仕事だけでなく周りの状況も気にするようになりました。貴社に入社できましたら、与えられた仕事を集中してこなすことはもちろん、周りの状況把握や他者へのサポートにも力を注いでいきたいと思います。', 1],
        ['私の強みはチームワーク力を強化する存在となれることです。学生時代、引っ越しのアルバイトをしていました。最初は入ったばかりでコツがつかめず、お客様の家の柱に物をぶつけ大変な迷惑をかけました。その後先輩が力の入れ方、持ち方、声かけの大切さを改めて教えてくださり、チームとして動くことを実感したのです。それ以降も先輩と話し合い傷をつけず安全な持ち方を研究し、新しい方法を提案することができました。このようにチーム一丸となってミスを無くし安全に行うことは、御社の研究開発部でも1番必要なことだと思います。私はこれからもチームワーク力で成果を求めていく姿勢で取り組んでいきたいです。', 1],
        ['私の強みは、チームワーク力を皆に伝え周りを巻き込む存在となれることです。学生時代、学園祭の実行委員長を務めた時に、最初は同じ立場の実行委員にどのような指示を出せば良いかわからず、そのせいでどんどん学園祭の準備が滞ってしまいました。しかし、昨年の実行委員長が、顧問としてバックアップしてくれたので、的確なアドバイスをもらいながら、一緒にやっていこうという雰囲気を自ら作り出しました。良い雰囲気のまま学園祭当日まで持っていき、無事に成功を収めることができました。このように、周りのみんなを巻き込む能力は御社の営業部において、なるべく多くの人からの知恵を得るために必要な能力であると確信しています。私は、これからもこのチームワークの力を皆に伝播させていく姿勢で、仕事に熱心に取り組んでいきたいと考えています。', 1],
        ['私の強みは、チームワークを作るためのきっかけとなることです。チームワークの基本は、お互いがお互いのことを良く知り、人間性をきちんと互いに理解し合うことだと私は考えています。大学の時のゼミナールでの経験から、多国籍の学生からなるチームでチームワークを作るために、飲み会や食事会を週に1度は実施することによって、メンバーとのコミュニケーションに努めました。最初は、なかなか話してくれないメンバーがほとんどでしたが、ひと月ほど同じことを続けていると、作業やレポートの作り方でこのほうが作りやすくていい、といった意見がでてくるようになり、それを活かすことができました。御社の仕事においても、同期や先輩方と最初はお互いのことを良く知らないとは思いますが、積極的に話しかけ、チームワークを作るための原点となり、仕事を盛り上げていきたいと考えております。', 1],
        ['何に対しても適応する能力があるのが、私の強みです。例えば、私が新しい業務を遂行する場合にはその業務の本質を学び、出来る限り迅速にその業務をおこなう能力を身につける努力をします。私は、さまざまな状況に即座に対応し適切に処理する能力があります。私が学生時代、アルバイトでファミリーレストランに勤めていたときに、先輩に指導して頂いたことは一度で覚えるようにし、聞き直したり間違った方法を取ったりすることも無いよう心がけました。ホール作業のみではなくキッチンやその他雑用をこなし、自ら率先して作業を探す努力を続けていました。この能力を生かして貴社では、まず出来るだけ早く仕事の本質を見極め、即戦力となる努力を致します。', 1],
        ['私の長所は、臨機応変に対応できる適応能力です。私は、高校生の頃からラップミュージックを中心に音楽活動を始めました。私はラップの歌詞を考えますが、より良い歌詞を作るために即興のラップバトルにも積極的に挑戦してきました。ラップバトルは即興で考えないといけないので、相手の特徴や癖、観客の雰囲気などに合わせて言葉を選んでいます。これは頭をフル回転させなくてはならず、アドリブで状況に応じた言葉を選ぶことが求められます。この経験のおかげでゼミの発表などでもあまり緊張することもなく、周りの様子を見ながら発言できるようになりました。このような、さまざまな状況に対応できる能力は、貴社で働く際にも活かせると思います。', 1],
        ['私は、何に対しても柔軟に対応できる適応能力があります。私には、大学に入ってから続けているアルバイトがあります。居酒屋のホール業なのですが、週末の夜など沢山のお客様が来店されます。ときにマニュアル通りの対応ができず、常に自分で判断をしなければなりません。グラスやお皿がすぐになくなってしまうときには、お客様の飲食するペースを見ながら、迷惑にならないように食器を下げます。トラブルの際にも、他のお客さんに迷惑がかからないよう、スタッフ同士で連携をとり解決してきました。貴社に採用された際には、私は自分の仕事だけでなく、周りのサポートもこなしながら、連携して業務を遂行していきたいと思います。', 1],
        ['私の強みは、人を巻き込む力です。大学では3Dアニメ制作のサークルに在籍していたのですが、なかなか全員納得できる作品を仕上げることができないでいました。そこで、実際にプロに聞てみること考え、あるコンテンツ制作会社の方とお会いしました。その方には「シナリオが良くない」という指摘を頂きました。プログラミングとキャラクターデザインには自信があるチームでしたが、確かにシナリオに時間を割いてきませんでした。そこで小説サークルの人にシナリオをお願いしたところ、最終的に素晴らしい作品を作ることができました。ここで学んだ、足りない力は人の力で補うことの大切さを、今後仕事にも活かしていきたいと思います。', 1],
        ['私のモットーは、人との繋がりを大切にすることです。私は、毎年夏休みに、農家である青森の祖父の家にしばらく滞在し、りんごの収穫を手伝います。しかし、収穫できるものは、りんごだけではありません。祖父は一人暮らしなのですが、収穫時期には私だけでなく、祖父の近所の人達もいつも手伝いに来てくれます。その方達にお話を伺うと、田舎では助け合いをしなければ生きていけないからと教えられました。子供の頃から続けているこの体験は、相手の立場を考えて自然と周りと協力する考え方を私に教えてくれ、多くの学びを収穫することができました。それは、今もゼミやサークル活動でメンバーに協力を募り目的を達成することに役立っています。', 1],
        ['私の強みは、団結精神です。高校時代、野球部ではグランド整備や、部室の掃除、ユニフォームの洗濯など、すべてマネージャーがしてくれていました。しかし、唯一のマネージャーが転校を機にいなくなってしまったことで、練習がしづらく、試合に負けることも多くなりました。そのため、私の提案で、1年生から3年生まで全員で分担して作業をやることを提案しました。先輩に言うのは、正直怖かったのですが、熱意を持って説得することで理解を得ることができました。その結果、練習がしやすくなり、試合にも勝てるようになりました、仕事でも周囲の人が気持ちよく仕事ができるよう、提案していきたいと思います。', 1],
        ['私の強みは、「縁の下の力持ち」として、自分の力を発揮できるところです。学生時代は長く演劇部で活動しており、私の主な活動は演出面などの裏方の仕事が主なものでした。全員でひとつの舞台を完成させるために、率先して部員の方向性をまとめることもありました。部員の結束も強まり、満足のいく舞台をおこなうことができました。御社に就職した際には、業務の裏方を積極的におこない、仲間全員で業務に取り組めるように地盤を固めてきたいと思っています。', 1],
        ['私の長所は「縁の下の力持ち」として、役割をしっかりとこなせることです。学生時代、私はテニス部のマネージャーとして部員たちを支えてきました。練習中のサポートなどの部員に関わる行動だけでなく、テニス部のPR活動として部員勧誘やブログの作成などもあわせておこないました。結果、部員からも「選手ではないが部に欠かせない存在」として評価されるようになりました。御社に就職した際にも、これまでの経験を活かして自分がおこなうべき役割をしっかりとこなし、表立って動く社員の補助をできるよう努めたいと思っています。', 1],
        ['私は、誰かをサポートすることにやりがいを感じる人間です。アルバイトとして家庭教師に携わっていた際、勉強のやり方が分からない子ども達のために勉強方法を考え、楽しく効率のいい勉強ができるようにサポートをおこないました。その結果子ども達の成績も上がり、勉強を進んでするようになったとご両親からも良い評価をいただきました。御社で就職した際には、自分の業務をこなすだけでなく周囲の社員のサポートもおこない、共に仕事の業績を伸ばしていけるように努めたいと思います。', 1],
        ['私の強みは周りと協力しながら積極的に取り組めることです。学生時代は機械科に在籍し、製図や機械設計、機械工作など、機械に関する専門的な知識を学びました。課題研究では、ソーラーカーを作成しました。作成では、回路の配線が上手くいかなかったり、思うようにソーラーカーが動かなかったりなど、多くの課題が発生しました。私はソーラーカーのチームリーダーとしてメンバーから出た意見をまとめ、積極的に自分から動くことを心がけました。多くの課題を解決してメンバーと改良を重ねた結果、想像していたものよりクオリティの高いソーラーカーを完成することができました。また、学校内でおこなわれた課題研究発表会では10チーム中1位になることができました。私は御社でも何事も積極的に取り組み、より良い製品づくりをして御社や世の中に貢献したいと考えています。', 1],
        ['私は体力があることが強みです。私は学生時代、新聞配達のアルバイトを2年間していました。毎朝3時に起床して、出勤後はチラシをはさんだり雨の日はビニールに新聞をいれたりしました。新聞配達では、時間内に配りきることや不配はないか常に心がけました。その結果、1年に1度おこなわれる総会では社内MVPをいただくことができました。バイクでの配達でしたが、毎日早起きして新聞配達を続けることで体力がつきました。また、部活動はサッカー部に所属していました。雨でグラウンドが使えない日は筋トレをしたり、部活動が休みの日は新聞の配達地区を走って体力向上に努めました。私は持ち前の体力を活かして、御社での部品組み立てやメンテナンスなどの仕事をこなしていきたいと考えています。', 1],
        ['私の強みは、何事も挑戦心を持って取り組めるということです。どのようなことでも挑戦心を持って取り組めば、困難な状況でも乗り越えられると考えています。私は2年間、カフェでキッチンのアルバイトをしていました。小さな個人経営のカフェだったのですが、近所に全国展開のカフェがオープンし、客足が途絶えたことがありました。そこで私は、新メニューの開発に挑戦しました。SNSで人気が出るように、色合いや可愛らしさを意識したデザートを考案しました。最初は店長から否定的な意見が出ていましたが、他のアルバイト生やお客さんの意見も取り入れ改良を重ねていったところ、新メニューとして発売されることになりました。発売後は徐々にSNSで人気が出始め、今では遠方からもお客さんがいらっしゃいます。私はこの挑戦心を活かして、貴社に入社しましたら商品企画部で新たな市場を切り拓いていきたいと考えております。', 1],
        ['私は、どのようなことも前向きに挑戦することができます。学生時代チアリーディング部に入っていたのですが、部員不足で廃部になるかもしれないという危機を迎えた経験があります。私はなんとか廃部だけは避けようと、勧誘のチラシ作成に挑戦しました。部への加入を強く訴えるチラシにしたかったので、パソコンのソフトを使った作成に挑戦しました。独学でソフトの使い方を習得し、写真なども挿入してインパクトのあるチラシに仕上げました。当初、他の部員達は廃部について仕方ないと言っていましたが、私の取り組みを見て、積極的にチラシの貼り出しに協力してくれるようになりました。全員が団結して勧誘活動をおこなったおかげで、10名の新しい仲間が増えました。私が貴社に入社しましたらこの挑戦心を活かし、広報として意欲的に商品をアピールしていきたいです。', 1],
        ['私の強みは、未経験の分野でも積極的に向き合う挑戦心です。私は文学部に所属しているのですが、商学部の友人が簿記の検定を取得したことで簿記に興味を持ちました。それまで、簿記に触れたことは一切ありませんでした。しかし、簿記は実生活に役立ちそうだと思い、参考書を買って勉強を始めました。勉強を進める内に検定に挑戦したい気持ちが湧き、それからは日商簿記検定3級の勉強に切り替え、毎日問題集を解きました。商学部の友人から答案のチェックをしてもらいながら対策をしたことで、1回目の検定で3級に合格することができました。私が貴社に入社しましたら、持ち前の挑戦心で営業部にて新規顧客獲得に貢献したいと考えております。', 1],
        ['私の強みは、困難なことに挑戦しそれを達成することです。大学2年生から中国経営を専門に研究するゼミナールに所属しましたが、そこは国際色豊かなゼミであり、全体の半分は外国からの留学生でした。中国からの留学生が最も多かったために、中国が日本語と合わせてゼミの公用語だった私は、コミュニケーションをとるために、中国語を学習し簡単な内容は中国語で意思疎通を図ることに挑戦しました。そのほうが、リーダーシップを取れるだろうと思った面もあります。最初は、まったくわからない状態でしたが、必死に勉強し中国語講座なども取って、ある程度話せるようになってきたときに周りの中国人留学生から、そこはこうしたほうがいい、ここはこう発音すると聞きやすい、といった点を教えてもらうことができました。この学習を続けた結果、今では日常会話程度なら中国語でできるようになりました。私は、この挑戦心と培った語学力を、御社の海外での営業かつ度に活用して仕事をしていきたいと考えております。', 1],
        ['私の強みは、挑戦を好み困難な目標を設定してそれに向かって努力することです。大学時代から商社に入りたいと思っていた私は、商社で仕事をしていくためには英語力が必須であることを痛感し、勉強しなければならないと感じました。そのため、英検準2級程度の能力しか持っていなかった私か、海外派遣最低ラインと言われるTOEIC730点を超えるためには、どうすればよいかを計画して受験し続けました。その結果、今この時点ではTOEIC760点をマークすることができましたが、まだ最低ラインを突破しただけなので、これからも800点を超えられるように目指して頑張っていきます。この、挑戦心を御社の海外での仕事や通訳に活用して、仕事に取り組んでいきたいと考えています。', 1],
        ['私は好奇心旺盛であり、新しいことでも臆さずに挑戦することができます。私は大学時代に居酒屋でアルバイトのキッチンでアルバイトをしていました。アルバイト先はチェーン店でしたが店舗ごとにオリジナルメニューを一品だけ提供していました。オリジナルメニューは毎月変わり、私は社員を手伝ってオリジナルメニューを一緒に考えていました。通常の業務をこなしながら新メニューを考えるのは大変でしたが非常にやりがいがあり、楽しく取り組むことができました。テレビやネットなどを参考に新メニューを提案し、それがメニューに載って多くのお客様に注文してもらうことができました。私は御社でも新しいことに挑戦して売上に貢献したいです。', 1],
        ['私は抑えることができない好奇心旺盛さがあります。好奇心旺盛なところを活かしてどんなことでも挑戦し、幅広く活躍することができます。私は大学時代サッカー部に所属していました。サッカーは小学生から続けており、さまざまなポジションを経験しました。一般的にはポジションは固定して練習しますが、私はたくさんのポジションに挑戦したいと思い、キーパー以外のポジション全てを経験しました。それぞれ違う能力を身に付けなければならず練習は大変でしたが、そのおかげで大きく成長することができ、最後の大会ではリーグ昇格という実績を残すこともできました。私は御社でもさまざまな部署を経験し、最終的に企業を引っ張る存在に成長したいです。', 1],
        ['私の長所は好奇心旺盛なところです。特に知的好奇心が強く、気になったことは納得がいくまでとことん調べることができます。私は大学時代経済学部に所属しており、ゼミでは地域の活性化についてを研究しました。図書館の文献を参考に研究を進めましたが、文献やネットだけではどうしても分からないこともあり、実際に現地に足を運んで真相を確かめました。現地に足を運ぶことで得られる情報は多くあり、文献では知ることのできない情報もありました。納得のいくまで研究を進めることで、研究発表を成功させることができました。私は知的好奇心を活かして御社でもさまざまなことを学び、幅広い知識を身に付けて成長したいと考えています。', 1],
        ['私は縁の下の力持ちとして周囲の人をサポートすることが強みです。私は学生時代、野球部のマネージャーをとして日々活動していたのですが、その中で選手が練習に集中できる、快適な環境づくりを第1に考え、取り組んでいました。選手の怪我が絶えなかったこともあり、私はスポーツ選手の怪我について多くの文献で勉強を行い、選手向けに怪我予防のための勉強会を実施していました。その結果、選手の怪我発生率は軽減し、選手から「ありがとう」と言われ、やりがいを感じました。私は入社後もよりよい職場環境づくりを念頭に置きながら貢献していきたいと考えております。', 1],
        ['私はサポートすることに喜びを感じる人間です。私は学生時代、女子テニス部の副部長として活動していたのですが、その中で部長をサポートすることに注力しました。部長ではなく副部長という立場であるからこそ、部長とは異なる視点で物事を客観的に見ることが可能になります。私は部長と部員の間を上手くつなぐことで、部長が1人で悩みを抱えないように部の問題を一緒に考えました。部長を含めた部員1人1人と向き合いサポートした結果、仲がいい部活と評判になりました。', 1],
        ['私の強みは縁の下の力持ちなところです。私は大学時代、文化祭の運営事務局としてステージイベントの裏方を行っていました。文化祭はどんなに事前準備を行ったとしても、当日何かしらのトラブルが起こるものです。そのため、その場その場での臨機応変な対応力が試されました。イベント当日、予定通りに発表グループが集まらないなどハプニングも多々ありましたが、その都度司会や音響と素早く連携を取り、イベントがスムーズに進むよう尽力しました。この経験から、状況を把握し、臨機応変な対応をする能力が身についたと考えています。', 1],
        ['私は人から負けず嫌いだと指摘されることがあります。小さい頃、運動会の徒競争などで一緒に走るクラスメイトに負けたとき、悔しくてたまらない気持になりました。私はその悔しさから誰よりも早く走りたいと、中学、高校と陸上部に籍を置き、走る技術を磨きました。その結果、高校でついに県大会の200m走競技で1位を取ることができたのです。負けず嫌いだからこそ成果をだすことができたと私は思っています。そしてそれは、私の長所でもあると信じています。この負けず嫌いという私の向上心をもって、是非とも御社の業務に寄与させていただきたいと思っています。どうぞよろしくお願いいたします。', 1],
        ['私は負けず嫌いです。しかし、それは私自身の長所であると考えています。コンビニでアルバイトしていた時、最初は経験不足からミスが多く店長から叱責されることも多かったのですが、他のスタッフからは入ったばかりだから仕方がないと言われていました。そこで私の負けず嫌いの気質に火が付き、シフトをたくさん入れできるだけ経験を積み、業務に慣れることに努めました。次第に余裕が生まれ、店の状態やお客様の表情をよく観察できるようになり、店長やスタッフからの評価が向上しました。 私の負けず嫌いは現状に満足することなく努力し続けられることです。', 1],
        ['私は、やると決めたら最後までやる「負けず嫌い」な人間です。学生時代にコンビニ店員のアルバイトをしていました。毎年2月になるとバレンタインチョコレートの販売が始まります。前年は80個の目標でしたが、今年は120個の目標。実際、残り1週間の予約数は40個ほどで、完売が厳しい状況でした。私はその中で何とか目標を達成したいと思い、チョコレートが売れる様子や目標数を店長の許可の下、SNSで公開しながら実況中継しました。購入者に写真撮影などを協力してもらいつつ、盛り上がる雰囲気を出そうと頑張りました。すると、他のスタッフも徐々に拡散に協力してくれました。朝の7時から23時まで、実況中継と販売を続け、他のスタッフと共に目標を達成しました。この負けず嫌いをもって、社会に出ても与えられた環境で最後まで頑張り、御社のために貢献できたらと考えております。', 1],
        ['負けず嫌いなところが自分の長所です。私は学生時代バスケットボール部に所属していましたが、体格に恵まれず、いつもベンチから試合を見守っていました。体格は自分ではどうすることもできませんが、テクニックは違います。部員が帰っても毎日一人で練習を行い、体育館を使える時間はひたすらドリブルとシュートを練習していました。その結果、ポイントガードとしてスタメン入りすることができました。御社でも、自分の置かれた環境に負けず、自分にできる努力を積み重ね、営業として業績を上げたいと思います。', 1],
        ['私は負けず嫌いな性格です。競争相手に負けないように、最大限の努力を惜しみません。大学時代、いつも順位を競っていた親友がいました。その親友に負けないように、単語帳や自作ノートの作成、教授への質問など、あらゆる手を尽くしました。親友もさらに勉学に励み、最終的に二人で学科の1位2位を争うようになりました。御社で営業部に配属された暁には、この負けず嫌いな性格を活かして仲間と切磋琢磨しあい、御社に貢献していきたいと思います。', 1],
        ['私は、一度決めたことはどんなに困難でもやり遂げます。一時期、資格取得に力を入れていたことがありました。「より実用的で就職にも大きく影響する資格を取りたい」と考え、多くの業界から需要が高い電気主任技術者3種を受験しようと決めました。しかし、私は当時電気に関する知識が乏しく、ほとんど一からの勉強であり、苦難の連続でした。しかし一度決めたことを途中で投げ出すことはせず、必死に勉強し、最終的に電気主任技術者を3回目の試験で取得することができました。御社でも諦めの悪さを活かし、試行錯誤しながら、より良い商品の開発に努めたいと思います。', 1],
        ['私は問題の根本原因を探り、その解決に向けて具体的な筋道を立てることができる人間である。ゼミで卒業論文の中間発表をした際、指導教官から多くの厳しい指摘をされ、構想を大きく変えなければならなくなった。卒業論文の評価は大学院入学試験の結果も左右するため、発表直後は落ち込み、焦りを感じた。しかし指摘されたいくつもの問題点を冷静に分析してみると、考察が甘い、あるいは的外れである根本的な原因は、基礎知識不足であるがゆえに先行研究を鵜呑みにしているからだと気づいた。知識は一朝一夕では身につかないが、残された時間は限られていたため、闇雲に参考文献を漁るのではなく、取捨選択した上で丁寧に文献を読み知識を蓄え、先行研究の正誤を見極める力をつけていった。その結果、自分の論文の矛盾点を省くと共に、新たに独自の視点からの考察を加えることができ、論文提出後の諮問ではその改良部分の面白みを評価してもらうことができた。', 0],
        ['遅刻しないのは、社会人として当たり前だと考えています。私は学生時代、早寝早起きを心がけていました。学業とバイトを両立させるためです。朝は7時からバイトだったので、毎朝5時に起きて仕度をして、毎日余裕を持って出勤していました。上司の方から『出勤のとき、いつも早いね』と褒められたこともあります。その結果、私は一度も遅刻することなく学業とアルバイトを両立できました。貴社には入れたら、学生時代に身に付けた早寝早起きの感覚で、誰よりも早く出社して仕事に取り組んでいきたいと考えています。', 0],
        ['私の売りはチーム活性化のために率先して動く行動力です。特にこの特徴が顕著に出ていたのはサークル活動で新入生歓迎活動の代表を任された時です。私の所属する広告サークルは毎年100名弱サークルに入会するため春の新入生歓迎活動に力を入れており、その役割は責任重大でした。その中でもフリーペーパーの作成は広報活動として重要なものであり、その認知度は常に学内最大級を誇っています。しかし、今回は人員不足からか作業過多が発生し、マネジメントを工夫する必要がありました。しかし、結果的には納期に間に合い、1500部の冊子を経由して180人の新入生を獲得することに成功しました。', 0],
        ['わたしは気配りがよくできる人間だとよく言われます。現在飲食店でアルバイトをしていますが、お客さんへの接客サービスは良く言われることが多く、ありがとうの言葉を度々いただきます。自らでこうされたらうれしい、便利かなと思うことを常に念頭に置いて、接客を行います。また、大学での部活動での打ち上げやイベントなどの幹事を行っており、お店選びや、内容などを大多数の希望をくみ上げ、みんなで楽しむ部活動をしてきました。この気遣いを仕事で積極的に取り入れ、自分の仕事だけでなく、いろんな人が気持ちよく仕事ができるようにしたいと思います。', 0],
        ['私の強みはコミュニケーション能力です。私は大学のサークルの練習参加率が悪かったことに問題意識を感じ、周りの部員全員（50人）とコミュニケーションを効果的にとり、楽しい雰囲気であるサークル環境を作り出しました。その結果、サークルの練習参加率は以前の2倍以上になり、その後も継続しております。またアルバイトでも同様に、多くのお客様とのコミュニケーションを通じて、他の社員に比べ1.5倍以上の売り上げを達成しました。以上より、私はコミュニケーション能力が強みであります。', 0],
        ['留学先に行った時、最初はなかなか英語で会話ができませんでした。日本では、周りの人に自分から話しかけるということはしていませんでしたが、留学先では積極的に自分から声をかけるようにしました。半年後には、英語での日常会話ができるようになり、日本に帰ってからも積極的に周りに話しかけられるようになりました。', 0],
        ['私はコツコツと努力ができる人間です。すぐに成果が出なくても、自身を奮い立たせ、コツコツと努力を積み上げ、目標を達成することが出来ます。具体的に、それを発揮したのがTOEICの点数を400点から800点にアップさせた経験です。将来は海外で働きたいという思いがあり、TOEICの学習を始めました。最初は全く結果が出ませんでしたが、必死に努力を継続した結果、上記の成果を達成出来ました。この姿勢を活かし、貴社でも活躍したいです。', 0],
        ['私は学生時代、営業系の会社でインターン生として働いていました。当初は同期の中でも営業成績は最下位でしたが、セミナーや書籍で知識を得たり、社内の先輩や上司の方々にアドバイスをもらったりして、着実に営業力を上げていきました。その結果、営業成績でトップになることができました。貴社の業務においても、この時の営業の経験を活かして、いち早く戦力となれるよう邁進したいと考えております。', 0],
        ['私の長所はゼミナール活動でメンバーをまとめ上げ、研究の成果にも繋げたことから、相手の立場に立って考えられる人間であると考えています。ゼミのメンバーからも、「1人ひとりに目が届いていてすごいね」と言われることからも、やはり相手の立場に立って考えられることが強みだと自負しています。ゼミナールで研究の成果に繋げられたことから、御社に入った際も、1人ひとりに気配りをしながら同じ目標へと向かっていきたいと考えています。', 0],
        ['私の長所は相手の立場になって考える事ができる点です。その例として、私が彼女と付き合った際にのことが挙げられます。私の彼女はお手洗いに行く頻度が高いです。なので彼女とデートに行く際に相手がお手洗いにいきたいかどうか、常に気を配っています。その結果、彼女には「気遣いができるね」と言われ以前よりも関係が良好になりました。このことから相手の気持ちに寄り添うことの大切さを学びました。御社に入った際にも、気配りという強みを活かして顧客の信頼を獲得したいと考えております。', 0],
        ['私の長所は協調性です。私は留学時、協調性を発揮して様々な国の人と交流してきました。そうする中で視野が広がり、他人の視点で物事を考えられるようになりました。貴社に入社できた際は、視野の広さと協調性を武器に多くのお客様に寄り添っていきます。', 0],
        ['私は、日々成長をモットーに周りの人・物・事からたくさんのことを吸収するように日々心がけています。本を一冊でも多く読む事や、人と少しでも多く会話する事で自己成長に繋がると考えているので、コミュニケーションを大切にしています。人見知りをしない、社交的な性格なので、自慢の「笑顔」で周囲を和ませたり、人間の基本として挨拶は必ず自分からします。問わず、臨機応変に対応できる事に自信があります。', 0],
        ['私は場の空気を良くすることができます。大学時代のゼミ活動では、機械学習について研究しており、主にPythonを使用して課題に取り組んできました。教授から与えられた課題は難しくゼミ室の空気もピリついていましたが、明るく振る舞うことで空気は和やかになりました。この強みを活かして、貴社でも活躍したいと考えております。', 0],
        ['私の長所は、粘り強いところです。私は学生時代、部活でバドミントンをやっていましたが、いわゆる弱小チームで、なかなか試合で勝つことができませんでした。他の部員の中には辞めていく人もいましたが、私はいつか必ず勝てるだろうと思い、何も考えずひたすら練習に励みました。その結果、最後の試合で隼決勝まで勝ち進むことができました。', 0],
        ['私の長所は、粘り強いところです。例えば人に何かを頼む時、一度や二度断られたぐらいでは諦めません。相手に「しつこい」と言われたとしても、なお頼み続けます。そのうち相手にも私の熱意が伝わり、「負けた」といって応じてくれます。この持ち前の粘り強さを、ぜひ貴社の営業の仕事に活かしたいと思っています。', 0],
        ['私は何事にも粘り強く取り組む性格です。小学5年生の時から続けている水泳でも、中々スピードが伸びない時にも諦めず、フォームや筋トレの内容などをコーチに相談したり自分でも調べたりして改良を重ね、中学2年で県大会優勝というレベルにまで到達しました。何事も諦めずに頑張れば、最後には結果が出せるという事を、この経験を通じて知ることができました。', 0],
        ['長期休暇を、いわゆるバックパッカー的に海外で過ごすのが趣味です。行き先は専らアジア圏ですが、乗り物の運賃や市場での買い物代金など、値段交渉により合意した額で支払うことが多々あります。私は、粘り強いです。徹底的に談判し、納得できるラインまで落とし込みます。掛け合いが長時間に及んでも決して譲りません。するとたいていは先方が根負けし、取り引きが成立します。これもアジア旅行の醍醐味のひとつです。希望している営業の仕事でも、この粘りを見せていきたいと思います。', 0],
        ['私は、目立たないことでも、地道に頑張り続ける努力を惜しみません。たとえ辛くても、必ず潜んでいる面白い面を見つけ出すのが上手で、そこに目を向けて乗り越えていく粘り強さがあります。施設管理のアルバイトに留まる理由を見失った際、投げ出さないための拠り処を探して、巡回時に周辺のゴミ拾いを始めました。自発的な取り組みがやがて面白味となり、快適だという利用者の声も聞こえてきました。この強みを活かせば、地味な作業も楽しくこなせると考えております。', 0],
        ['私は結果を出すまで諦めずに取り組むことができます。私は家庭教師のアルバイトをしているのですが、その生徒は中間テストでは学年の最下位層の順位でした。そこで私はわかりやすく教えるために様々な工夫をし、次の学期の期末テストでは上位20位に入るまでに学力を伸ばすことに成功しました。生徒本人からもご両親からも感謝され、諦めずに取り組むことの大切さを私自身も学ぶことができました。この諦めない強さを御社でも発揮していきたいと考えています。', 0],
        ['私は責任感があると周囲の人から言われます。責任感は社会人にとって最も重要なもののひとつだといえます。時間を守る、〆切を守る、途中で投げ出さないというのは、学生ならば責任感があると言われますが、社会人なら当たり前のことです。しかし、この当たり前のことすらできなくては同僚や上司、取引先などから信頼されなくなってしまいます。私は持ち前の責任感で信頼される営業として御社に貢献していきたいと考えています。', 0],
        ['私の強みは、チーム全体の士気を上げ、チームのパフォーマンスを上げる存在となれることです。私は学生時代に、バドミントンのサークルに所属していました。平日は、毎日大学の体育館を借りて練習をしていました。しかし、試合にはなかなか勝つことはできませんでした。その理由は、本気で練習に取り組んでいる人が少なかったからです。大学のサークルなので、本気でバドミントンに取り組んでいる人もいれば、遊び程度に考えて、基礎練習を疎かにする人もいます。このままでは全体の士気が下がり続け、試合に勝つことなんてできないと考えた私は、おおまかに「本気で勝とうと練習している人」と「楽しくバドミントンをしたい人」に分け、それぞれ練習メニューを変更しました。その結果、バドミントンサークルに所属している人の誰もが楽しく、充実したサークル活動を行うことができるようになり、試合にも勝つことができるようになりました。貴社に入社した際には、私の強みを活かし、チーム全体のパフォーマンスを上げていきたいと考えています', 0],
        ['私は学生時代に、バイトリーダーとしてチームを取りまとめていました。接客業ですが、バイト仲間の中には接客が苦手で、なかなか笑顔になれない人もいました。このままではいけないと思い、私はバイトの仲間たちに積極的に話しかけるようにしました。仕事上必要なことはもちろん、趣味や好きな音楽、テレビ番組のことなどを仕事の休憩時間や仕事終わりに話すように心がけました。その結果、チーム全体の仲が深まり、報連相がうまくいくようになったり、お客様への態度にも変化があったりしました。チーム全体をまとめ、パフォーマンスを上げることが私の強みのひとつだと考えています。貴社ではこの強みを活かし、積極的にチームの仲間に話しかけ、全体のパフォーマンスを上げていきたいと思います。', 0],
        ['私の強みは、チームワーク力があることです。その強みを活かせたのは、洋服販売のバイトをしていた時のことです。お客様の要望を聞きながら洋服を販売するため、基本的には他のスタッフと必ずしも協力する必要はありません。しかし私は、より売上を伸ばすためにチームワークを重視するようにしました。その結果、売り上げが伸び、表彰を受けることができました。私は貴社でも、持ち前のチームワーク力を活かし、営業成績を伸ばしていきたいと思います。', 0],
        ['私は、人に合わせることがとても上手です。これは協調性という適応能力のひとつであると考えています。そしてこの適応能力が私の最大の強みです。２年間アルバイトとして居酒屋のホールスタッフをしていますが、スタッフ同士が仲良く仕事をすることを私は心がけています。忙しいときには、スタッフ同士がいらついて険悪になりかけることがありますが、私はできる限り平静を装い波風を立てないようにしています。状況を判断し、私の方から謝るなど、お客様に問いに各迷惑をかけないよう、精いっぱい自分を制御することにしているのです。このような協調的な性格は貴社の仕事をおこなう上でも大変役立つに違いありません。', 0],
        ['私は学生時代、カラオケ店でアルバイトを経験し、2つの対応力を身につけました。私はカラオケ店でキッチン・飲食物の運びを担当し、混雑時や業務が重複したときには、常に「今やるべきこと」を意識し仕事に取り組みました。またドリンク・フードを作る業務では、最初は仕事が遅く周囲に度々迷惑をかけていましたが、メニューを全て覚え、どのようなオーダーに対しても素早く対応できるようになりました。繁忙時にはドリンク作りや運びなど、できる限り丁寧かつ迅速に作業することを心がけていました。このようなことを通して身に付けた対応力「プライオリティ」と「正確性」と「迅速性」を仕事に生かし、貴社に貢献したいと考えています。', 0],
        ['どのような変化にも対応できる適応能力が、私の長所です。私は居酒屋でアルバイトをしていましたがそこでの作業はほとんどマニュアル化されており、効率的に仕事ができるようになっていました。しかし私は、あえてさらに効率的に仕事ができるよう提案をしました。オーダーを取りに行ったあと帰ってくるとき、直接帰ってきていたものを、客席を回って必ず何かひとつ仕事をしてから帰るよう提案しました。こうすることで効率的な仕事ができるようになり評価を受けました。変化を恐れず提案できるところが私の長所です。この長所を活かし貴社でも、変化を恐れずなにごとにも提案していきたいと考えます。それが貴社の業績に貢献できると思います。', 0],
        ['私は、よくトラブルを仲裁することができます。大学入学と同時に始めた飲食店のアルバイトでは、同じ店舗で長く務め上げ、仕事も認めてもらいフロアリーダーまで経験させて頂きました。しかし、その職場では、よく従業員同士のトラブルが発生していました。そのため、私が間に入り話し合いの場を設け、お互いの言い分を聞き、妥協点を見つけ解決に導くように努力しました。その時の経験から、さまざまな立場の人をまとめ、協力して仕事をすることの難しさと大切さを知りました。そこで学んだことを活かし、どんな時でもチームで力を合わせることを目標としています。', 0],
        ['私の強みは、「人のために役に立ちたい」という思いが強いことです。普段から世話好きで、サークル仲間には「後輩の面倒見がいい」とよく言われます。これには理由があり、高校時代のバレー部で、厳しくも丁寧な指導をして頂ける、ある一人の先輩に出会ったことがきっかけです。先輩は、なかなかレギュラーになれず悩んでいた私に声をかけ、毎日夜遅くまで練習に付き合ってくださいました。おかげで、最終的には全国大会にまで出場することができ、それ以来、私は常に周りに気を配り、困っている人がいれば話を聞き、何かを頼まれればできるだけ協力するようにしています。これからも多くの人の役立てるよう、努力していくつもりです。', 0],
        ['私の持ち味は、笑顔のムードメーカーです。常日頃から笑顔は絶やさず、場の空気を大切にし、仲間やチームと、いつも笑顔で仕事ができることを目標としています。私は在学中、飲食店のアルバイトを約4年間続けてきました。主に厨房での調理スタッフを担当していたのですが、大変だった点は、ランチタイムの時間帯などに厨房の中がピリピリとした雰囲気になることです。そのような時こそ、まずは私から率先して笑顔を絶やさないようにすることで、場の雰囲気を和ませることができました。これから先の仕事においても、場の雰囲気を大切にして、チームワークを大切にしたいと考えています。', 0],
        ['私の強みは、縁の下の力持ちとして行動できるところです。学生時代は野球部のマネージャーとして、選手達のサポートをすることが多くありました。また、サポートだけでなく、練習方法を提案したり、新人勧誘の場で部員達の代わりに積極的に前に出て部員を増やせるように努めました。結果として部員も増え、野球部が気持ちよく試合ができるようになりました。御社に就職した際には、他の社員をサポートして業務を円滑に進められるように努めたいと思っております。', 0],
        ['私の強みは、目立たない所で他のスタッフの支援をおこなうのが得意なことです。私はアルバイトをしていた際、食品関係の仕事に携わっていました。主に内勤で、上司の指示に従ってお客様に美味しい料理を提供できるよう、他のスタッフを支援していました。その結果、店舗の売上も上がり良い評価を得られるようになりました。御社に就職した際には、目立たない仕事でも真面目に取り組み、他の社員を支援していけるように努めたいと思っております。', 0],
        ['私はあちこち動き回ることがあまり得意ではありません。大学におけるゼミ活動などでも、現地に行って色々と聞き込みを行い、調査を行う機会がありますが、そのようなことに対して苦手意識を持っています。そのようなことよりも黙々と作業を行うほうが自分には合っていると思い、そのため製造業で働くことを志望しております。その中で御社は、自動車業界で圧倒的なシェアを誇っており、御社であれば安定して長く働き続けることが出来ると思い志望しております。', 0],
        ['私は「黙々と作業を行うことが得意」であることと、「英語という強みを持っている」ことから海外に拠点を持つ製造業の企業への就活を検討しており、その中で御社の求人を見つけました。御社はアメリカ向けの輸出に力を入れており、海外拠点の展開も今後積極的に進めていく方針だということを知りました。そのような環境であれば、自身の強みを活かしながら働くことが出来ると思い、御社を志望しております。小さな頃から運動が苦手で体力面で若干不安を感じてはいますが、精一杯仕事に邁進したいと考えております。', 0],
        ['私は、目の前の仕事に黙々と取り組むことが出来る点で、製造業を志望しております。私は学生時代の経験から、細かな作業を集中して黙々とこなしていくことが得意であり、やりがいを感じています。製造業であれば、自身の目の前の業務に集中していればよく、余計な気を使わなくて良い点に魅力を感じております。その中で私が御社を志望する理由は、雰囲気に魅かれたからです。御社の説明会に参加し、一人一人の就活生に丁寧に対応する社員の姿を見て、このような方々と共に働き、自らもそのような対応が出来るように成長していきたいと考えております。', 0],
        ['私の強みは、挑戦心を持っているところです。学生時代は様々な資格を取得するために常に勉強し、大学の講義などと並行して試験を受けていました。これらの挑戦の結果として様々な資格を取得することができ、知識や経験として身に付けることができました。御社に就職した際には、この挑戦心をもって様々な新しいことに挑戦したいと思っています。それにより、企業に貢献できるよう心がけたいと思います。', 0],
        ['私のアピールしたい長所は、どんなことにでも常に挑戦していく部分です。アルバイトでは、基本の仕事に慣れてくると様々な職務を任されるようになり、中には未経験の内容もたくさんありました。アルバイトの職務内容は全てが初めての経験だったので、どのような仕事にも常に挑戦する気持ちで取り組んでいました。結果として、アルバイト店員の中でも頼りにされるようになりました。企業に就職した際には、新しい職務や事業にも常に挑戦することを忘れず取り組みたいと思っています。', 0],
        ['私の強みは何事にも諦めずに挑戦するところです。学生時代から長く陸上を続けていましたが、ある時期から記録が伸びなくなってしまいました。いろいろな訓練方法を試すなど改善しようとしたのですが、もう無理だと感じることが多かったです。それでも諦めずに訓練を続け、長い時間をかけて記録を伸ばしました。企業に就職した際には、結果の有無に関わらず新しい仕事に取り組み、常に挑戦する気持ちを忘れずに職務に関わりたいと思っています。', 0],
        ['私は大学時代にバックパッカーとして世界中を旅し、さまざまな経験をしました。見たことのない風景、聞いたことのない街を訪ねて、そこでさまざまな文化に触れました。多くの人が知っている国であっても街によっては全く知られていない場所も多く、私はできるだけ知らない場所を求めて旅を続けました。バックパッカーの経験を通してさまざまなことを知り、私の中で財産になっています。私は好奇心旺盛さを活かして御社でもさまざまな仕事に挑戦し、そこでの経験を財産として成長していきたいです。', 0],
        ['私の長所は好奇心旺盛なことです。好奇心を活かしてさまざまな物事に取り組み成長することができます。私は大学時代に居酒屋でアルバイトをしていました。もともとはキッチンとして働いていましたが、ホールの仕事にも興味があり、ホールの経験もしました。また営業中のシフトだけではなく、営業前のシフトにも興味がありましたので、そのシフトにも入り、仕込みの作業も覚えました。私はさまざまな仕事に挑戦することができますので、その行動力を活かして御社でもたくさんの仕事を経験し、成長したいと考えています。', 0],
        ['私の長所は好奇心旺盛であることです。興味のあることには果敢に挑戦し、そこで成長することができます。私は、大学時代にさまざまなアルバイトを経験しました。居酒屋や道路作業員、販売員や工場勤務などさまざまなアルバイトを経験し、それらでさまざまな能力を身に付けることができました。アルバイトのメリットはすぐに辞めることができることなので、私は大学時代に多くの仕事を知ろうと思い、気になるアルバイトを見つけては挑戦していました。その結果、幅広い知識やスキルを身に付けることができ、それらの能力を御社でも活躍したいと考えています。', 0],
        ['私は縁の下の力持ちです。私はこの強みをこれまでさまざまな場面で活かしてきました。高校の部活動では副将として、部長と各部員の連携を強めるべく奔走しました。また、大学のテニスサークルでは、外務として、他大学との調整を円滑に進め、対校戦などの運営に貢献しました。このように私は、誰も見ていないところであっても、困っている人を助けたり努力を続けたりすることができる人間なのです。この強みを活かして、貴社においても業務に励みたいと考えております。', 0],
        ['私の強みは縁の下の力持ちであるところです。私は学生時代、サッカー部に所属し、日々活動していたのですが、なかなかレギュラーに定着することが出来ませんでした。私の所属していたサッカー部ではレギュラーか否かに関わらず、試合当日には必ず部員全員で応援に行くルールとなっていたため、レギュラーから外されてしまった際、活躍するレギュラーの姿を見て、悔しい思いを多くしました。しかし、自身の気持ちを必死に抑えて応援に徹し、チームの雰囲気がよくなるよう一生懸命努力しました。', 0],
        ['私の持ち味は縁の下の力持ちである性格です。私は学生時代、イベントスタッフのアルバイトをし、さまざまなコンサートの運営に携わりました。その中で、上司からの指示に従い、黙々と仕事をこなし、ステージを成功へと導くことにやりがいを感じました。私は昔から目立つ行動をすることが好きではなく、自身が中心になって物事を進めていくことに関して苦手意識はありますが、指示された業務などはコツコツと一生懸命やる自信があります。', 0],
        ['私はとにかく他の人に負けることが許せない性格です。高校までの勉強は教科書を中心にしたものですから、元来記憶ものに強い私は問題なく好成績を取れていました。しかし大学の成績評価にはレポート、論文といった新たなものが加わります。講義でレポート課題が出題されるたびに傑作を書き上げよう、最高評価を取ろうと意気込みました。しかし、私より高い評価を得るものがいると、私は我慢ができなくなります。なんとしても勝ちたいのです。私より前に人がいるのが大嫌いな性格と言えます。努力をしてゼミで一番の成績を修めました。私の努力が実を結んだため、御社でもこのように努力をして頑張っていこうと思っています。', 0],
        ['私は負けず嫌いさが自慢です。小学校の時から野球のチームに入っており、高校では県予選のベスト8、そして大学はスポーツ推薦で入学しました。私は人一倍努力しましたので、いつもレギュラーで活躍していました。高校や大学では全国大会には出場できませんでしたが、それは私が悪いわけではなく他のメンバーのエラーで負けてしまったからです。しかし、私はくじけずに一人で練習を続けました。この負けず嫌いさは御社でも必ず役に立てると思います。', 0],
        ['私は負けず嫌いな性格です。昔からどんなことでもクラスで1番になりたくて勉強も運動も頑張ってきました。おかげで高校は進学校に入学できましたが、今度は周りも優秀な人ばかりなので、成績で1番を取れなくなってしまいました。しかし親からは何でもいいから1番になりなさいと言われていましたので、趣味であるテレビゲームの対戦で頑張ることにしました。技なども必死で練習し、ネットでの対戦は負け無しになるようになりました。', 0],
        ['私は子どもの頃からずっと海外に興味を持っていました。海外が好きだったので高校では世界史を選択しました。高校生の頃には大学入学後にできるだけ多くの国に行こうと考え、資金集めのためにアルバイトもしていました。大学入学後は夏休みと冬休みに必ず海外旅行に行き、3年間で10ヶ国以上訪れました' ,0],
    ]

valid_data = [
        ['私の強みは、チャレンジ精神です。大学時代は洋服店でアルバイトをしていました。私はエクセルを使ったことがなかったのですが、勉強になると思い自ら在庫管理の仕事を志望しました。在庫管理の仕事は大変でしたが、学ぶことが多かったです。在庫管理の他にも、仕入れの仕事にも興味が出て、短期間でしたが任せてもらえました。仕入れの仕事も在庫管理と同じで大変でしたが、貴重な経験ができました。御社でも持ち前のチャレンジ精神で色々と取り組みます。', 0],
        ['私の強みはコミュニケーション能力です。私は居酒屋でアルバイトをしているのですが、どのお客様とも仲良く話すように心がけています。また、他の従業員と会話することも心がけています。従業員同士年齢が近いので、学校の話や趣味の話で盛り上がります。学校や趣味の話をすると自分が知らない世界が知れるので、とても刺激になっています。学校生活でも毎日誰かと話すようにしているので、コミュニケーション能力には自信があります。この強みを活かして御社でも活躍したいと思っています。', 0],
        ['私の強みは目標達成力があるところです。アルバイト先の先輩から「簿記を取っておくと役に立つ」と言われたので、日商簿記検定3級を取得しました。アルバイトとの両立は大変でしたが、上手く工夫することで勉強を続けることができました。結果を出すことは仕事においても重要だと考えるので、御社に入社後も目標達成力を発揮して行きたいと思います。', 0],
        ['私の長所は、毎日こつこつと努力を続けられる、最後まで諦めないところです。高校生の頃、吹奏楽部に所属しており、フルートのパートリーダーを任されていました。あまり人の先頭に立って引っ張ることは得意ではありませんでしたが、他のパートリーダーの人達の良いところを取り入れ、パートメンバーの意見をしっかりと聞くように努めました。このような努力を地道に続けた結果、メンバーからの信頼を得ることができ、練習中にもよく相談を受けるほど頼りにされるようになりました。この経験を通して得た、最後まで地道に努力を続けられるという資質を活かし、貴社でも売上目標を明確に立てて貢献したいと考えています。' , 0],
        ['人の話をしっかりと受け止め、共感することで信頼関係を築くことができます。大学時代に所属していたラクロス部では、副キャプテンとして多くの部員の悩みを聞くように努めてきました。今何を考え、どうしたいのか、どんな悩みを抱えているのかを話し合い、解決策を一緒に考えることで少しずつ信頼関係を築いてきました。また、自分一人が話を聞くだけではなく、ミーティングを何度も重ねることで部員全員がお互いの理解を深めることができるように努めてきました。当初はバラバラだった部員の意識も、こうした活動を通して一つにまとまることができ、信頼し合えるチームが出来上がったと思います。この経験から、相手の話を聞くことの大切さや、それによって信頼関係が出来上がっていくことを学ぶことができました。仕事をする際にもお客様や社内の人たちの話を聞き、共感することで信頼関係を築いていきたいと思います。', 0],
        ['私の強みは辛いことも諦めずに立ち向かう粘り強さです。わたしは大学の時にコンビニエンスストアでアルバイトをしていました。コンビニでは担当する業務が思ったより多く、なかなかすべてを覚えることができなかったので、最初は店長やアルバイトの先輩に叱られてばかりでした。何度も辞めてしまいたいと思うことがありましたが、せっかく始めたからには諦めたくないという気持ちが強く、もっと前向きに仕事に取り組もうと考えました。そこで、家に帰ってからもアルバイト先で学んだことをしっかり復習し、どの仕事も任せてもらえるような知識を身に付けました。また、気持ちよくお客様に接する事を心掛けて、どんなに疲れている時でも、どんなに忙しい時でも笑顔を絶やさないようにしてきました。今では店長や他のアルバイトの人にも認めてもらえるようになり、そのアルバイトを続けていくことができています。これからもどんな仕事も諦めずに取り組む姿勢を大事にしたいです。', 0],
        ['友人と「やったことのないことに挑戦」するイベントを企画・運営するサークルを立ち上げ、企画部の部長に任命されました。この役割を務める際に、部員の主体性を高めるために尽力しました。なぜなら、参加者や部員が楽しめるイベント作りには、多くの意見が必要だと思ったからです。しかし当初は意見が出ず、私の意見ばかりが採用されていたので、この状況を打破する努力をしました。具体的には、話し合いの場を盛り上げて発言しやすい雰囲気を作り、それでも発言できない子には強要せず、個人的に意見を聞いて会議に反映させました。さらに個々人を観察して、長所に合わせた役割分担をしました。その結果、メンバーの意欲が高まり、活発に意見が飛び交うようになりました。それに伴い、回数を重ねるごとに参加者を増やすこともできました。以上の経験から、周りを見ながら人の立場に立って考えることの大切さを学び、努力は報われることの感動を味わいました。', 0],
        ['私は、アルバイト先での雰囲気作りに力を入れました。私が入ったばかりの頃は少しピリピリした雰囲気がありました。これでは仕事も円滑に行うことができないので、私は積極的にスタッフへコミュニケーションをとることを心がけました。飲み会などイベントを開催して交流深める機会を設けたり、年の離れた方へ積極的に仕事の相談をするようにしたところ、徐々に何でも話せる雰囲気ができ、今ではアルバイト先で起こったことは全員で共有し解決に導くことができるようになりました。', 0],
        ['私のアピールポイントは、学生としての本分である学業に対して、もちまえの向上心を発揮できる点です。私は学生時代、誰よりも一生懸命に向上心をもって勉学に励み、大学ではよい成績をおさめることができました。体調がすぐれない日や、モチベーションが保てずに、なまけたいと思うこともありましたが、毎日努力を欠かしませんでした。この毎日の勉強のおかげで、いまの私があります。', 0],
        ['私には、あきらめずに努力を続ける力があります。サークルの勧誘で、先輩から目標30名の勧誘を依頼されましたが、もっとがんばればもっと勧誘できると考え、自分で目標値をさらに上げました。そして、勧誘のために効果的なフライヤーを作成し、地道な努力でたくさんのフライヤーを配布しました。それだけではなく、説得できそうな学生はあらかじめ目星をつけておき、直接説得を行い、多くのメンバーの勧誘に成功することができました。先輩からは、多くのサークルメンバーを集めたことで、高い評価をいただいたうえに感謝されたことで、高く目標をかかげてよかったと感じました。', 0],
        ['私の長所は、向上心があるところです。特にテストでは学年のTOP10に入る目標を立て、小学生の頃からずっと努力してきました。夜寝る前や朝に、必ず予習復習を欠かさずするようにしました。高校生の時、中間テストでずっと順位を競っていたライバルに負けてしまったので、期末テストでは必ず勝とうと必死で勉強しました。先生から単語をもっと暗記すれば苦手科目が克服できるとアドバイスを受け、自分のやってきた勉強法に組み込んでやり方を改善しました。その結果、得意科目だけでなく苦手科目でもライバルに勝つことができました。そこで満足せずにさらに努力を続け、ついに学年1位を取るまでになりました。この経験のように、目標達成のために柔軟にやり方を見直しながら努力し、達成後も新たな目標を定め、常に向上心を持って貴社に貢献したいと考えています。', 1],
        ['私の長所は負けず嫌いであることです。大学時代はテニスサークルに所属しており、サークル活動とアルバイトのかけもちで、学業と両立させられずに成績が振るわない時期がありました。原因は時間配分がうまくできていない点だと思い、スケジュール帳を独自に作成しました。それぞれに集中する時間と休息時間などを細かく定め、その通りに行動するようにしたところ学年10位以内の成績を取れるようになっただけでなく、サークルで出た大会でもベスト4に入ることができました。最初から完璧にこなせず、失敗することもあると思います。それでも私は、自分の長所である負けず嫌いをフルに活用して、より目標達成に向けて効率的な方法を常に考えながら、企画チームのサポートができる存在を目指し、努力を続けたいと思っています。', 1],
        ['私の長所は、毎日こつこつと努力を続けられる、最後まで諦めない負けず嫌いなところです。高校生の頃、吹奏楽部に所属しており、フルートのパートリーダーを任されていました。あまり人の先頭に立って引っ張ることは得意ではありませんでしたが、他のパートリーダーの人達の良いところを取り入れ、パートメンバーの意見をしっかりと聞くように努め、週に1度ミーティングで意思疎通を図るようにしました。このような努力を地道に続けた結果、メンバー間の信頼関係を築くことができ、全パートの中で一番の演奏ができるようになりました。そしてついに、メインパートの演奏を任された大会で金賞を取ることができました。この経験を通して得た、最後まで地道に努力を続けられるという資質を活かし、貴社でも売上目標を明確に立てて地道に営業を続けることで、貢献したいと考えています。', 1],
        ['私は継続力があり、一度始めたことは最後まで諦めずにやり遂げることが出来ます。大学時代は文化祭の実行委員になり、キャンパス内の一部分の企画展示の責任者になりました。文化祭の実行委員になったのは、高校時代に大学の文化祭に行った際、企画展示に感動して、自分もやりたいと思ったのがきっかけでした。私は、展示を見ている人に感動を与えることを目標に立てました。そして、参加者や協力者への呼びかけを行い、何枚も企画書を作り直し作業に没頭しました。他の実行委員の意見を聞きながら何度もブラッシュアップすることで最高の展示ができ、周囲からの感謝も多数得られました。御社でも一度掲げた目標を簡単に諦めず、粘り強く取り組み続けることで活躍したいと考えています。', 1],
        ['私は、どれだけ困難な状況でも、それを乗り越えるための行動力があります。IT企業のインターンシップに参加した際、社内で使われている業界用語が理解できずに苦しむ日々が続きました。そこで私はメモ帳とパソコンを常に持ち歩くようにし、分からない言葉はその場ですぐに検索することを実践しました。知らないことをそのままにするのではなく、解決するためにはどうすれば良いのかを考えた結果です。最終日の会議では、ほとんどの内容が理解できるようになりました。私はこの経験を活かし、営業という仕事でも様々な苦難を乗り越えていきたいと考えています。', 1],
        ['私は、目標達成に向けて計画を立て、それを実現する行動力があります。私は大学で英語を専攻し、それ以外でも英会話スクールに通い、日々語学力の向上に努めています。さらに語学力を磨くため、3年次には1年間の留学を経験しました。その際には、毎日の学習計画を立て、学校の勉強と自宅学習をしながら、現地の人とも積極的に交流しました。その結果、語学力が向上しただけでなく、広い視野で物事が見えるようになりました。御社では、留学で身に付けた語学力を活かすだけでなく、あらゆる視点から考えて、業績に貢献できればと考えています。', 1],
        ['私の強みは分析力の高さです。私は大学時代、個別指導の塾講師としてアルバイトをしていました。塾には勉強に苦手意識を持つ生徒が多かったので、1人1人の生徒に楽しんでもらえるように苦手な教科の分析とともに、性格に合った授業の進め方ができるよう生徒のタイプを分析しました。10人の生徒を受け持っていたので個別に分析してプログラムを作成するのは大変でしたが、目に見えて生徒の成績が上がったり、塾が楽しいと言ってもらえたりすることにやりがいを感じました。効果があった生徒については、効果があらわれた要因を調べるための分析も忘れずにおこないました。御社に入社後も自ら課題を見つけ分析し、よりよい結果を求めていきたいです。', 1],
        ['私の強みは、チームを成功に導くリーダーシップです。2年間居酒屋でアルバイトをしていたのですが、1人がした失敗をまた他の誰かがするという連鎖が続いていました。私はスタッフ間のコミュニケーションが上手くいっていないことが原因だと考え、仕事のマニュアルやミス手帳を掲載したスタッフノートを作成しました。シフトで働く時間が違っても、ノートであれば休憩時に誰でも気軽に見たり書き込めたりできます。失敗が多く起きている業務の内容だけでなく、お客様に褒められたことなどを書き、スタッフのモチベーションアップにも活用しました。この結果、仕事のミスが軽減され、現場でもスタッフ同士で助け合うことが増えました。御社でも、チーム全員がお互いに支え合えるような働き方をしていきたいです。', 1],
        ['私は毎日必ず2時間ずつ勉強し、1年間で730時間勉強してきました。なまけたいと感じた日もありましたが、日本と海外をつなぐ架け橋になりたいという将来の目標を掲げていたので、自分自身に甘えを許さずやりきることができました。努力が実を結んだ結果、学期末には25教科中23教科で優評価をいただき、学年でトップの成績を残しています。このことから、高い目標を達成し、自分自身をポジティブに評価することができる喜びを得ました。仕事においても自主努力を積み重ね、高く設定した目標にもくじけることなく取り組むことができると考えております。', 1],
        ['私には、高い目標にむかって努力を惜しまない向上心があります。サークルの勧誘で先輩から目標30名の勧誘を依頼されましたが、勧誘方法を工夫すれば、もっと勧誘できると考え、自分で目標値をプラス20名の50名に設定しました。勧誘のために、いままでサークルで誰も行ったことがなかった、効果的なキャッチコピーやイラストをいれたフライヤーを作成しました。毎朝大学の校門前に立ち、1週間で合計2,000枚のフライヤーを配布しました。それだけではなく、学生の空き時間や放課後に直接説得を行い、目標の50名の勧誘に成功することができました。このことから、あえて目標値を高く設定しても、地道な努力と創意工夫を重ねれば、本来の目標よりもよい結果を残すことができることを体感しました。御社においても、与えられた目標のさらに上をめざし、努力と創意工夫をもって達成していきたいと考えております。', 1],
    ] 

In [5]:
df = pd.DataFrame(data, columns=['text', 'label'])
print(df['label'].value_counts())
print(df['text'].nunique())

1    56
0    47
Name: label, dtype: int64
103


In [6]:
valid_df = pd.DataFrame(valid_data, columns=['text', 'label'])
print(valid_df['label'].value_counts())
print(valid_df['text'].nunique())

1    10
0    10
Name: label, dtype: int64
20


In [9]:
config_path = 'drive/My Drive/bert_wiki_ja/bert_config.json'
checkpoint_path = 'drive/My Drive/bert_wiki_ja/model.ckpt-1400000'
bert = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True, trainable=False, seq_len=512)

KeyboardInterrupt: ignored

In [0]:
maxlen = 512

sp = spm.SentencePieceProcessor()
sp.Load('drive/My Drive/bert_wiki_ja/wiki-ja.model')

def get_tokens(text):
  indices = np.zeros(maxlen, dtype = np.float32)

  tokens = []
  tokens.append('[CLS]')
  tokens.extend(sp.encode_as_pieces(text))
  tokens.append('[SEP]')

  for t, token in enumerate(tokens):
    try:
        indices[t] = sp.piece_to_id(token)
    except:
        logging.warn(f'{token} is unknown.')
        indices[t] = sp.piece_to_id('<unk>')

  return indices

In [0]:
X = []
for text in df['text']:
  X.append(get_tokens(text))

y = df['label'].values
y = utils.np_utils.to_categorical(y)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

X = [X, np.zeros_like(X)]
# X_train = [X_train, np.zeros_like(X_train)]
# X_test = [X_test, np.zeros_like(X_test)]

In [0]:
X_valid = []
for text in valid_df['text']:
  X_valid.append(get_tokens(text))

y_valid = valid_df['label'].values
y_valid = utils.np_utils.to_categorical(y_valid)


X_valid = [X_valid, np.zeros_like(X_valid)]

In [0]:
model_path =  'drive/My Drive/bert_wiki_ja/bert_check_point.model'

inputs = bert.inputs[:2]
bert_output = bert.get_layer('Encoder-12-FeedForward-Norm').output
x1 = Bidirectional(LSTM(256))(bert_output)
outputs = Dropout(0.2)(x1)
outputs = Dense(units=64)(outputs)
outputs = Dense(units=2, activation='softmax')(outputs)

model = Model(inputs, outputs)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc', 'mse'])

history = model.fit(
    X, y, 
    validation_data=(X_valid, y_valid),
    epochs=50,
    batch_size=3,
    shuffle=True,
    verbose=1,
    callbacks=[
               EarlyStopping(patience=5, monitor='val_acc', mode='max'),
               ModelCheckpoint(monitor='val_acc', mode='max', filepath=model_path, save_best_only=True)
    ])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Train on 103 samples, validate on 20 samples
Epoch 1/50
103/103 [==============================] - 87s 849ms/step - loss: 1.2960 - acc: 0.5049 - mean_squared_error: 0.3932 - val_loss: 0.6407 - val_acc: 0.7500 - val_mean_squared_error: 0.2242
Epoch 2/50
103/103 [==============================] - 82s 796ms/step - loss: 0.6049 - acc: 0.7087 - mean_squared_error: 0.2075 - val_loss: 0.6255 - val_acc: 0.6500 - val_mean_squared_error: 0.2184
Epoch 3/50
103/103 [==============================] - 82s 794ms/step - loss: 0.5400 - acc: 0.7670 - mean_squared_error: 0.1808 - val_loss: 0.5903 - val_acc: 0.7500 - val_mean_squared_error: 0.2012
Epoch 4/50
103/103 [==============================] - 82s 799ms/step - loss: 0.4949 - acc: 0.7961 - mean_squared_error: 0.1558 - val_loss: 0.5698 - val_acc: 0.7000 - val_mean_squared_error: 0.1908
Epoch 5/50
103/103 [==============================] - 83s 801ms/step 

In [10]:
model_path =  'drive/My Drive/bert_wiki_ja/bert_check_point.model'
model = load_model(model_path, custom_objects=get_custom_objects())

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
test = '私は、目標達成に向けて計画を立て、それを実現する行動力があります。私は大学で英語を専攻し、それ以外でも英会話スクールに通い、日々語学力の向上に努めています。さらに語学力を磨くため、3年次には1年間の留学を経験しました。その際には、毎日の学習計画を立て、学校の勉強と自宅学習をしながら、現地の人とも積極的に交流しました。その結果、語学力が向上しただけでなく、広い視野で物事が見えるようになりました。御社では、留学で身に付けた語学力を活かすだけでなく、あらゆる視点から考えて、業績に貢献できればと考えています。'
test_tokens = [get_tokens(test)]
test_tokens = [test_tokens, np.zeros_like(test_tokens)]
model.predict(test_tokens)[0][1]

0.7864954

In [11]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 512)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 512, 768), ( 24576000    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 512, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________